In [1]:
import numpy as np
import pandas as pd

/Users/nimishasingh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [4]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82
1,0,95,2.0,2.0,2,1,1226.0,1.0,0.0,0.0,1,2,0.95
2,0,103,2.0,2.0,1,1,1000.0,0.0,0.0,0.0,1,0,0.32
3,0,99,3.0,4.0,4,3,1615.0,1.0,0.0,1.0,0,2,1.60
4,0,5,2.0,2.0,1,3,582.0,0.0,1.0,0.0,0,2,0.48


In [5]:
df.shape

(3554, 13)

In [6]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [7]:
X = df.drop(columns=['price'])
y = df['price']

In [8]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer

In [9]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [10]:
# Applying the log1p transformation to the target variable for converting into Normal distributed 
y_transformed = np.log1p(y)

In [11]:
y_transformed 

0       0.598837
1       0.667829
2       0.277632
3       0.955511
4       0.392042
5       2.140066
6       1.098612
7       1.064711
8       0.788457
9       1.704748
10      0.936093
11      0.955511
12      1.178655
13      1.327075
14      0.788457
15      0.936093
16      0.737164
17      0.765468
18      1.018847
19      0.615186
20      0.641854
21      0.788457
22      0.470004
23      2.302585
24      2.079442
25      0.641854
26      0.336472
27      1.894617
28      1.147402
29      1.686399
30      0.494696
31      1.350667
32      1.504077
33      0.530628
34      0.932164
35      2.224624
36      1.960095
37      1.131402
38      0.810930
39      0.530628
40      1.335001
41      1.871802
42      1.275363
43      1.386294
44      0.810930
45      1.131402
46      1.238374
47      0.867100
48      0.996949
49      1.047319
50      0.470004
51      0.593327
52      1.887070
53      0.916291
54      0.896088
55      0.974560
56      0.371564
57      0.300105
58      1.9095

In [12]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [13]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['property_type', 'bedRoom', 'bathroom',
                                  'built_up_area', 'servant room',
                                  'store room']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['sector', 'balcony', 'agePossession',
                                  'furnishing_type', 'luxury_category',
                                  'floor_category'])])

# Apply Linear Regression

In [14]:
# creating a Pipeline

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [15]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [16]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [17]:
scores.mean()

0.8558125370136732

In [18]:
scores.std()

0.015558416578484056

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [20]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
y_pred = np.expm1(y_pred)

### MSE score for Linear Regression

In [23]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.6483880981407997

It indicates that the model is making a prediction error of 64 lakhs. In practical terms, if the actual price of a property is 1 crore, the model is predicting it to be 1.64 crores, resulting in a deviation of 64 lakhs. This signifies the amount by which the model's predictions differ from the true values, highlighting the extent of the prediction inaccuracy.

# Apply Support Vector Regressor (SVR)

In [24]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [26]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [25]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [28]:
scores

array([0.89662402, 0.89877479, 0.89792723, 0.87771211, 0.88355941,
       0.88927068, 0.88628532, 0.89646795, 0.84999741, 0.86874181])

In [29]:
scores.mean()

0.8845360715052788

In [30]:
scores.std()

0.014784881452419995

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [32]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [33]:
y_pred = pipeline.predict(X_test)

In [34]:
y_pred = np.expm1(y_pred)

In [35]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5324591082613247

- so after applying SVR reduce MAE score by 11 lakh 